In [1]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
import json
import unicodedata
import datetime

In [2]:
url_general = 'https://www.factcheck.org/the-factcheck-wire/'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [3]:
num_pags = int(soup_general.find_all('span', attrs={'class' : 'sr-only'})[1].text)

print(num_pags)

328


In [4]:
def replace_characters(text):
    text_aux = ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', '', text_aux.lower().replace(u'\xa0', u' ').replace(u'\n', u' ')).split(" ")

In [5]:
def replace_characters_title(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')

In [6]:
bulos = {}

cont = 0
for i in range(1, num_pags+1):
    url = 'https://www.factcheck.org/the-factcheck-wire/page/' + str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for h3 in soup.find_all('h3',attrs={'class' : 'entry-title'}): 
        a = h3('a')[0]
        title = a.text
        link = a.get('href', None)
        words = dict()
        
        html_new = requests.get(link, allow_redirects=False).text
        
        
        
        soup_new = BeautifulSoup(html_new, 'html.parser')
        title_list = re.sub(r'[^\w\s]', '', title.lower()).split(" ")
        
        for w in title_list:
            if w in words:
                words[w] += 1
            else:
                words[w] = 1
                
        new_text = soup_new.find('div',attrs={'class' : "entry-content"})
        
        # Obtener la fecha
        
        date = soup_new.find(class_="entry-date published updated")
        for d in date:
            new_date = datetime.datetime.strptime(str(d), "%B %d, %Y")
        
        formated_date = new_date.strftime('%Y/%m/%d')
        
        for text in new_text('p'):
            for w in replace_characters(text.get_text()):
                if w in words:
                    words[w] += 1
                else:
                    words[w] = 1
        
        bulos[cont] = {"titulo" : replace_characters_title(title), "link" : link, "date" : formated_date, "words_count" : words}
        cont += 1

In [10]:
#bulos

In [8]:
len(bulos)

3274

In [9]:
with open('fakenewsFactCheck.json', 'w') as fp:
    json.dump(bulos, fp)